# Parsbert 

### Libraries

In [1]:
! pip install transformers
! pip install nltk
! pip install hazm
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
! wget https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml -P ./data
! wget https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip -P ./data
! unzip ./data/doc_collection.zip -d ./data
! wget https://raw.githubusercontent.com/sobhe/hazm/master/hazm/data/stopwords.dat -P ./data

--2021-12-27 13:59:37--  https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50854 (50K) [text/plain]
Saving to: ‘./data/evaluation_IR.yml’

evaluation_IR.yml   100%[===================>]  49.66K  --.-KB/s    in 0.01s   

2021-12-27 13:59:37 (3.92 MB/s) - ‘./data/evaluation_IR.yml’ saved [50854/50854]

--2021-12-27 13:59:37--  https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/language-ml/2-LM-embedding-proj

### Read data

set path of data in `PATH` variable

In [3]:
PATH = './data/IR_dataset/'
YAML_PATH = './data/evaluation_IR.yml/'
YAML_PATH = YAML_PATH.rstrip('/')
PATH = PATH.rstrip('/')

### Clean Text

In [4]:
import codecs
import hazm
import re

In [6]:
stopwords = [x.strip() for x in codecs.open('./data/stopwords.dat','r','utf-8').readlines()]
normalizer = hazm.Normalizer()

def remove_spaces_reg():
    # return space patterns
    space_pattern = r"[\xad\ufeff\u200e\u200d\u200b\x7f\u202a\u2003\xa0\u206e\u200c\x9d]"
    return re.compile(space_pattern)

def remove_symbols_reg():
    # return symbol patterns
    symbol_patterns = r"(\d|\"|'ٍ|¬|[؛“،,”‘۔’’‘–]|[|\.÷+\]\[\)\(\:\-\?»\=\{}\*«»_…\؟!/ـ]|[۰'ٓ۫'ٔ]|[ٓٔ]|[ًٌٍْﹼ،َُِّ«ٰ»ٖء])"
    return re.compile(symbol_patterns)

def remeove_arabic(text):
    # remove arabic alphabet
    mapping = {
        u"ۀ" : u"ه",
        u"ة" : u"ت",
        u"ي" : u"ی",
        u"ؤ" : u"و",
        u"إ" : u"ا",
        u"ٹ" : u"ت",
        u"ڈ" : u"د",
        u"ئ" : u"ی",
        u"ﻨ" : u"ن",
        u"ﺠ" : u"ج",
        u"ﻣ" : u"م",
        u"ﷲ" : u"",
        u"ﻳ" : u"ی",
        u"ٻ" : u"ب",
        u"ٱ" : u"ا",
        u"ڵ" : u"ل",
        u"ﭘ" : u"پ",
        u"ﻪ" : u"ه",
        u"ﻳ" : u"ی",
        u"ٻ" : u"ب",
        u"ں" : u"ن",
        u"ٶ" : u"و",
        u"ٲ" : u"ا",
        u"ہ" : u"ه",
        u"ﻩ" : u"ه",
        u"ﻩ" : u"ه",
        u"ك" : u"ک",
        u"ﺆ" : u"و",
        u"أ" : u"ا",
        u"ﺪ" : u"د"
    }

    arabic_keys =  re.compile(r"(" + "|".join(mapping.keys()) + r")")
    return arabic_keys.sub(lambda x: mapping[x.group()], text)


spaces_reg = remove_spaces_reg() 
symbols_reg = remove_symbols_reg()


def clean_text(text, allspace=True, punc=True, stopw=True):
    if allspace:
        text = re.sub(spaces_reg, " ", text)
    if punc:
        text = re.sub(symbols_reg, "", text)

    text = remeove_arabic(text)
    text = normalizer.normalize(text)
    
    if stopw:
        regex = r"\b(?:" + "|".join(map(re.escape, stopwords)) + r")\b"
        text = re.sub(regex, " ", text)
        
    text = re.sub("(\s)+", " ", text)
    text = text.strip()
    return text

store txt files into a list named `doc`

In [7]:
from nltk.tokenize import sent_tokenize

docs = []
for index in range(0, 3258):
    with open(f"{PATH}/{index}.txt", 'r', encoding='utf8') as file_reader:
      doc = file_reader.read()
      # doc = sent_tokenize(doc)
      docs.append(doc)

In [8]:
docs = [clean_text(doc, stopw=False) for doc in docs]
len(docs)

3258

In [9]:
import yaml

In [10]:
evaluation = {}
with open(YAML_PATH, "r") as stream:
    try:
        evaluation = dict(yaml.safe_load(stream))
    except yaml.YAMLError as exc:
        print(exc)

### Check GPU Availability

In [11]:
import torch
torch.cuda.is_available()

True

### Implementation

In [12]:
top_k = 100

In [13]:
!pip install -qU sentence-transformers

     |████████████████████████████████| 1.2 MB 4.3 MB/s 


In [14]:
from sentence_transformers import models, SentenceTransformer, util

In [15]:
def load_st_model(model_name_or_path):
    word_embedding_model = models.Transformer(model_name_or_path)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False)
    
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    return model

In [16]:
# Load the Sentence-Transformer
model = load_st_model('m3hrdadfi/bert-zwnj-wnli-mean-tokens')

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/451M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/747k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [17]:
corpus_embeddings = model.encode(docs, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/102 [00:00<?, ?it/s]

In [18]:
queries = evaluation.keys()

In [19]:
result = []
for query in queries:
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()
    result.append(cos_scores)

In [20]:
#We use torch.topk to find the highest 5 scores
top_results_indices = []
top_results_values = []
for cos_score in result:
  top_results_indices.append(torch.topk(cos_score, k=top_k).indices.numpy())
  top_results_values.append(torch.topk(cos_score, k=top_k).values.numpy())

In [21]:
print("Sentence embeddings:")
print(model.parameters())

Sentence embeddings:
<generator object Module.parameters at 0x7f2fbdf92b50>


In [22]:
evaluation['آدولف هیتلر  شکست و مرگ']['relevant'][0]

341

In [23]:
!pip install import-ipynb
import import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=fd553e07c3ccbaa6e12dc3b4e37461ad9871d639ac1c597a66fa362cbb74d4a4
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [24]:
import evaluation as ev

importing Jupyter notebook from evaluation.ipynb


In [25]:
mrr = []
for i,key in enumerate(evaluation.keys()):
  mrr.append(
      ev.mean_reciprocal_rank([(top_results_indices[i] == evaluation[key]['relevant'][0]).astype(int)])
  )
sum(mrr)/len(mrr)

0.25262212570561055

### Pooling Methods

In [ ]:
# Max Pooling - Take the max value over time for every dimension. 
def max_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    token_embeddings[input_mask_expanded == 0] = -1e9  # Set padding tokens to large negative value
    return torch.mean(token_embeddings, 1)[0]

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

#### Fine Tune Model

In [26]:
sents = []
for doc in docs:
  sents.append(nltk.sent_tokenize(doc))

In [27]:
! pip install stanza

     |████████████████████████████████| 432 kB 3.8 MB/s 
     |████████████████████████████████| 170 kB 37.0 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=3f2f7b690bc5811cbf47a6c45fb3d7e880f604e76e3a26d6088c0e3603011d5b
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [28]:
import stanza
stanza.download('fa')
nlp = stanza.Pipeline(lang='fa', processors='tokenize')

2021-12-27 14:11:30 INFO: Downloading default packages for language: fa (Persian)...


2021-12-27 14:11:37 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-12-27 14:11:37 WARNING: Language fa package default expects mwt, which has been added
2021-12-27 14:11:37 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |

2021-12-27 14:11:37 INFO: Use device: gpu
2021-12-27 14:11:37 INFO: Loading: tokenize
2021-12-27 14:11:37 INFO: Loading: mwt
2021-12-27 14:11:37 INFO: Done loading processors!


In [29]:
bag = list(set([item for doc in docs for item in nlp(doc).sentences]))
bag_size = len(bag)
bag_size

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


3286

In [30]:
import random

sentence_a = []
sentence_b = []
label = []

for sentence in docs:
  print(sentece)
  num_sentences = len(sentence)
  if num_sentences > 1:
      start = random.randint(0, num_sentences-2)
      # 50/50 whether is IsNextSentence or NotNextSentence
      if random.random() >= 0.5:
          # this is IsNextSentence
          sentence_a.append(sentence[start])
          sentence_b.append(sentence[start+1])
          label.append(0)
      else:
          index = random.randint(0, bag_size-1)
          # this is NotNextSentence
          sentence_a.append(sentence[start])
          sentence_b.append(bag[index])
          label.append(1)

In [32]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('m3hrdadfi/bert-zwnj-wnli-mean-tokens')

In [33]:
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

ValueError: ignored

In [ ]:
inputs.keys()

In [ ]:
inputs['labels'] = torch.LongTensor([label]).T

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-6)

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
torch.cuda.empty_cache()
loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 10

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        print(input_ids.shape)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))